<a href="https://colab.research.google.com/github/soadD/netflix_data/blob/main/tabela_global_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Extração**

In [1]:
#Abertura das bibliotecas
import pandas as pd
import numpy as np

In [2]:
# Passo 4: Extração da informação via pandas
df_global = pd.read_excel('/Users/ewerton/Documents/dev/projeto_final_soulcode/all-weeks-global.xlsx')

/Users/ewerton/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
df_global.head()

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10,is_staggered_launch,episode_launch_details
0,2024-05-26,Films (English),1,Atlas,NaN,56300000,2.0000,28200000.0,1,False,NaN
1,2024-05-26,Films (English),2,Thelma the Unicorn,NaN,17400000,1.6333,10700000.0,2,False,NaN
2,2024-05-26,Films (English),3,Ice Age: Dawn of the Dinosaurs,NaN,15100000,1.5833,9500000.0,1,False,NaN
3,2024-05-26,Films (English),4,Mother of the Bride,NaN,14000000,1.5000,9300000.0,3,False,NaN
4,2024-05-26,Films (English),5,Disturbia,NaN,14100000,1.7500,8100000.0,2,False,NaN


In [10]:
df_global.describe()

,weekly_rank,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10
count,6080.000000,6.080000e+03,2000.000000,2.000000e+03,6080.000000
mean,5.500000,1.836791e+07,3.595500,4.558100e+06,3.090296
std,2.872518,2.620671e+07,3.046722,5.035917e+06,3.212275
min,1.000000,7.000000e+05,0.000000,6.000000e+05,1.000000
25%,3.000000,6.220000e+06,1.666700,1.800000e+06,1.000000
50%,5.500000,1.140000e+07,2.150000,3.000000e+06,2.000000
75%,8.000000,2.040000e+07,5.125025,5.125000e+06,4.000000
max,10.000000,5.717600e+08,23.366700,5.080000e+07,30.000000


## **Pré-Análise**

In [8]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        6080 non-null   object 
 1   category                    6080 non-null   object 
 2   weekly_rank                 6080 non-null   int64  
 3   show_title                  6080 non-null   object 
 4   season_title                2959 non-null   object 
 5   weekly_hours_viewed         6080 non-null   int64  
 6   runtime                     2000 non-null   float64
 7   weekly_views                2000 non-null   float64
 8   cumulative_weeks_in_top_10  6080 non-null   int64  
 9   is_staggered_launch         6080 non-null   bool   
 10  episode_launch_details      78 non-null     object 
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 481.1+ KB


In [20]:
for column_name in df_global.columns:
    print(50*"-")
    print(df_global[column_name].value_counts())
print(50*"-")

--------------------------------------------------
week
2024-05-26    40
2022-05-29    40
2022-07-17    40
2022-07-10    40
2022-07-03    40
              ..
2023-06-04    40
2023-05-28    40
2023-05-21    40
2023-05-14    40
2021-07-04    40
Name: count, Length: 152, dtype: int64
--------------------------------------------------
category
Films (English)        1520
Films (Non-English)    1520
TV (English)           1520
TV (Non-English)       1520
Name: count, dtype: int64
--------------------------------------------------
weekly_rank
1     608
2     608
3     608
4     608
5     608
6     608
7     608
8     608
9     608
10    608
Name: count, dtype: int64
--------------------------------------------------
show_title
Manifest                  51
Stranger Things           50
Money Heist               33
You                       31
Bridgerton                30
                          ..
Full Swing                 1
In Love All Over Again     1
Lyle, Lyle, Crocodile      1
Thunivu 

In [14]:
unique_show = df_global['show_title'].unique()

In [15]:
for title in unique_show:
    print(title)

Atlas
Thelma the Unicorn
Ice Age: Dawn of the Dinosaurs
Mother of the Bride
Disturbia
Sing 2
Ice Age: Collision Course
A Simple Favor
Madame Web
Security
Golden Kamuy
In Good Hands 2
Crew
The Courier
Monster
Laapataa Ladies
Shaitaan
In Good Hands
4 Kings 2
The Tearsmith
Bridgerton
Ashley Madison: Sex, Lies & Scandal
Baby Reindeer
Jurassic World: Chaos Theory
Tires
Queen Charlotte: A Bridgerton Story
Bodkin
Buying London
The 8 Show
Demon Slayer: Kimetsu no Yaiba
Thank You, Next
Franco Escamilla: Ladies' man
The Asunta Case
The Atypical Family
Heeramandi: The Diamond Bazaar
Queen of Tears
Garouden: The Way of the Lone Wolf
Maestro in Blue
Shrek
Upgrade
Sonic the Hedgehog
The Boss Baby
Unfrosted
Smile
Partiu América
Honeymoonish
The Guardian of the Monarchs
Adagio
Tad, The Lost Explorer and the Emerald Tablet
Kevin Hart: The Kennedy Center Mark Twain Prize for American Humor
The Roast of Tom Brady
A Man in Full
Blood of Zeus
Cooking Up Murder: Uncovering the Story of César Román
Kaiju No.

In [16]:
len(unique_show)

1993

In [23]:
import requests

def get_movie_genre(api_key, title):
    url = f"http://www.omdbapi.com/?apikey={api_key}&t={title}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'Genre' in data:
            return data['Genre']
        else:
            print(f"Genre not found for title: {title}")
            return ""
    else:
        print(f"Error: Unable to fetch data (status code: {response.status_code})")
        return ""
    

In [18]:
title_gender_dict: dict = {}

In [19]:
for title in unique_show:
    title_gender_dict[title] = ""

In [20]:
title_gender_dict

{'Atlas': '',
 'Thelma the Unicorn': '',
 'Ice Age: Dawn of the Dinosaurs': '',
 'Mother of the Bride': '',
 'Disturbia': '',
 'Sing 2': '',
 'Ice Age: Collision Course': '',
 'A Simple Favor': '',
 'Madame Web': '',
 'Security': '',
 'Golden Kamuy': '',
 'In Good Hands 2': '',
 'Crew': '',
 'The Courier': '',
 'Monster': '',
 'Laapataa Ladies': '',
 'Shaitaan': '',
 'In Good Hands': '',
 '4 Kings 2': '',
 'The Tearsmith': '',
 'Bridgerton': '',
 'Ashley Madison: Sex, Lies & Scandal': '',
 'Baby Reindeer': '',
 'Jurassic World: Chaos Theory': '',
 'Tires': '',
 'Queen Charlotte: A Bridgerton Story': '',
 'Bodkin': '',
 'Buying London': '',
 'The 8 Show': '',
 'Demon Slayer: Kimetsu no Yaiba': '',
 'Thank You, Next': '',
 "Franco Escamilla: Ladies' man": '',
 'The Asunta Case': '',
 'The Atypical Family': '',
 'Heeramandi: The Diamond Bazaar': '',
 'Queen of Tears': '',
 'Garouden: The Way of the Lone Wolf': '',
 'Maestro in Blue': '',
 'Shrek': '',
 'Upgrade': '',
 'Sonic the Hedgehog'

In [24]:
# Replace 'your_api_key' with your actual OMDb API key
api_key = '46ff1df3'

for title, value in title_gender_dict.items():
    print(50*"-")
    print(f"title: {title}, value: {value}") 
    genre = get_movie_genre(api_key, title)
    print(f"The genre of '{title}' is: {genre}")
    title_gender_dict[title] = genre

--------------------------------------------------
title: Atlas, value: 
The genre of 'Atlas' is: Action, Adventure, Drama
--------------------------------------------------
title: Thelma the Unicorn, value: 
The genre of 'Thelma the Unicorn' is: Animation, Adventure, Comedy
--------------------------------------------------
title: Ice Age: Dawn of the Dinosaurs, value: 
The genre of 'Ice Age: Dawn of the Dinosaurs' is: Animation, Adventure, Comedy
--------------------------------------------------
title: Mother of the Bride, value: 
The genre of 'Mother of the Bride' is: Comedy, Drama, Romance
--------------------------------------------------
title: Disturbia, value: 
The genre of 'Disturbia' is: Crime, Drama, Mystery
--------------------------------------------------
title: Sing 2, value: 
The genre of 'Sing 2' is: Animation, Adventure, Comedy
--------------------------------------------------
title: Ice Age: Collision Course, value: 
The genre of 'Ice Age: Collision Course' is: Ani

The genre of 'Kaiju No. 8' is: Animation, Action, Adventure
--------------------------------------------------
title: Dr. Stone, value: 
The genre of 'Dr. Stone' is: Animation, Action, Adventure
--------------------------------------------------
title: Barbarian, value: 
The genre of 'Barbarian' is: Horror, Mystery, Thriller
--------------------------------------------------
title: Secrets of the Neanderthals, value: 
Genre not found for title: Secrets of the Neanderthals
The genre of 'Secrets of the Neanderthals' is: 
--------------------------------------------------
title: The Judge, value: 
The genre of 'The Judge' is: Crime, Drama, Mystery
--------------------------------------------------
title: The Frozen Ground, value: 
The genre of 'The Frozen Ground' is: Biography, Crime, Drama
--------------------------------------------------
title: Woody Woodpecker Goes to Camp, value: 
The genre of 'Woody Woodpecker Goes to Camp' is: Animation, Adventure, Comedy
--------------------------

The genre of 'The Hijacking of Flight 601' is: Drama, History, Thriller
--------------------------------------------------
title: Anna, value: 
The genre of 'Anna' is: Action, Thriller
--------------------------------------------------
title: The Bricklayer, value: 
The genre of 'The Bricklayer' is: Action, Thriller
--------------------------------------------------
title: Hotel Transylvania 2, value: 
The genre of 'Hotel Transylvania 2' is: Animation, Adventure, Comedy
--------------------------------------------------
title: Damsel, value: 
The genre of 'Damsel' is: Action, Adventure, Fantasy
--------------------------------------------------
title: The Super Mario Bros. Movie, value: 
The genre of 'The Super Mario Bros. Movie' is: Animation, Adventure, Comedy
--------------------------------------------------
title: Amar Singh Chamkila, value: 
Genre not found for title: Amar Singh Chamkila
The genre of 'Amar Singh Chamkila' is: 
--------------------------------------------------
ti

The genre of 'Homicide' is: Crime, Mystery, Thriller
--------------------------------------------------
title: Avatar The Last Airbender, value: 
The genre of 'Avatar The Last Airbender' is: Animation, Action, Adventure
--------------------------------------------------
title: Gabby's Dollhouse, value: 
The genre of 'Gabby's Dollhouse' is: Animation, Adventure, Comedy
--------------------------------------------------
title: Buying Beverly Hills, value: 
The genre of 'Buying Beverly Hills' is: Reality-TV
--------------------------------------------------
title: Hot Wheels Let's Race, value: 
The genre of 'Hot Wheels Let's Race' is: Animation, Action, Adventure
--------------------------------------------------
title: Dave Attell: Hot Cross Buns, value: 
Genre not found for title: Dave Attell: Hot Cross Buns
The genre of 'Dave Attell: Hot Cross Buns' is: 
--------------------------------------------------
title: Iron Reign, value: 
The genre of 'Iron Reign' is: Drama, Mystery, Thriller


The genre of 'The Tourist' is: Action, Thriller
--------------------------------------------------
title: Maamla Legal Hai, value: 
Genre not found for title: Maamla Legal Hai
The genre of 'Maamla Legal Hai' is: 
--------------------------------------------------
title: House of Ninjas, value: 
The genre of 'House of Ninjas' is: Action, Drama, Thriller
--------------------------------------------------
title: Ruby Gillman, Teenage Kraken, value: 
The genre of 'Ruby Gillman, Teenage Kraken' is: Animation, Action, Comedy
--------------------------------------------------
title: Dune, value: 
The genre of 'Dune' is: Action, Adventure, Drama
--------------------------------------------------
title: Ballerina, value: 
The genre of 'Ballerina' is: Action, Thriller
--------------------------------------------------
title: Through My Window 2: Across the Sea, value: 
Genre not found for title: Through My Window 2: Across the Sea
The genre of 'Through My Window 2: Across the Sea' is: 
---------

...

Error: Unable to fetch data (status code: 401)
The genre of 'The Girl in the Spider's Web' is: None
--------------------------------------------------
title: Amina, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Amina' is: None
--------------------------------------------------
title: Nobody Sleeps in the Woods Tonight 2, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Nobody Sleeps in the Woods Tonight 2' is: None
--------------------------------------------------
title: Monk Comes Down the Mountain, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Monk Comes Down the Mountain' is: None
--------------------------------------------------
title: The Trip, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'The Trip' is: None
--------------------------------------------------
title: Maya and the Three, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Maya and the Three' is: None
----------

Error: Unable to fetch data (status code: 401)
The genre of 'Sounds Like Love' is: None
--------------------------------------------------
title: Thalaivii, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Thalaivii' is: None
--------------------------------------------------
title: Luccas Neto em: O Mapa do Tesouro 2, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Luccas Neto em: O Mapa do Tesouro 2' is: None
--------------------------------------------------
title: Lucifer, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Lucifer' is: None
--------------------------------------------------
title: Dave Chappelle: The Closer, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Dave Chappelle: The Closer' is: None
--------------------------------------------------
title: Scaredy Cats, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Scaredy Cats' is: None
----------------------------------

Error: Unable to fetch data (status code: 401)
The genre of 'Invasion' is: None
--------------------------------------------------
title: The Secret Diary of an Exchange Student, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'The Secret Diary of an Exchange Student' is: None
--------------------------------------------------
title: Out of my league, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Out of my league' is: None
--------------------------------------------------
title: Lassie Come Home, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Lassie Come Home' is: None
--------------------------------------------------
title: The 100, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'The 100' is: None
--------------------------------------------------
title: Octonauts: Above & Beyond, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Octonauts: Above & Beyond' is: None
--------------

Error: Unable to fetch data (status code: 401)
The genre of 'The Losers' is: None
--------------------------------------------------
title: The Last Letter From Your Lover, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'The Last Letter From Your Lover' is: None
--------------------------------------------------
title: Wish Dragon, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Wish Dragon' is: None
--------------------------------------------------
title: World Trade Center, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'World Trade Center' is: None
--------------------------------------------------
title: Top Secret UFO Projects: Declassified, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Top Secret UFO Projects: Declassified' is: None
--------------------------------------------------
title: Hunter X Hunter (2011), value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Hunter X Hunt

Error: Unable to fetch data (status code: 401)
The genre of 'The Mitchells vs. The Machines' is: None
--------------------------------------------------
title: Murder by the Coast, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Murder by the Coast' is: None
--------------------------------------------------
title: Rurouni Kenshin: Origins, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Rurouni Kenshin: Origins' is: None
--------------------------------------------------
title: Prime Time, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Prime Time' is: None
--------------------------------------------------
title: Sophie: A Murder in West Cork, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Sophie: A Murder in West Cork' is: None
--------------------------------------------------
title: Katla, value: 
Error: Unable to fetch data (status code: 401)
The genre of 'Katla' is: None


In [25]:
title_gender_dict

{'Atlas': 'Action, Adventure, Drama',
 'Thelma the Unicorn': 'Animation, Adventure, Comedy',
 'Ice Age: Dawn of the Dinosaurs': 'Animation, Adventure, Comedy',
 'Mother of the Bride': 'Comedy, Drama, Romance',
 'Disturbia': 'Crime, Drama, Mystery',
 'Sing 2': 'Animation, Adventure, Comedy',
 'Ice Age: Collision Course': 'Animation, Adventure, Comedy',
 'A Simple Favor': 'Comedy, Crime, Mystery',
 'Madame Web': 'Action, Adventure, Sci-Fi',
 'Security': 'Action, Crime, Thriller',
 'Golden Kamuy': 'Animation, Action, Adventure',
 'In Good Hands 2': '',
 'Crew': 'Comedy, Drama',
 'The Courier': 'Drama, History, Mystery',
 'Monster': 'Biography, Crime, Drama',
 'Laapataa Ladies': 'Comedy, Drama',
 'Shaitaan': 'Horror, Thriller',
 'In Good Hands': 'Comedy, Drama, Romance',
 '4 Kings 2': 'Action, Crime, Drama',
 'The Tearsmith': 'Drama, Fantasy, Romance',
 'Bridgerton': 'Drama, Romance',
 'Ashley Madison: Sex, Lies & Scandal': 'Documentary, Crime, History',
 'Baby Reindeer': 'Biography, Drama

In [40]:
# Define a custom function to categorize values
def categorize_gender(x):
    return title_gender_dict[x]

In [43]:
df_global['gender'] = df_global['show_title'].apply(categorize_gender)

In [44]:
df_global.head(20)

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10,is_staggered_launch,episode_launch_details,gender
0,2024-05-26,Films (English),1,Atlas,NaN,56300000,2.0000,28200000.0,1,False,NaN,"Action, Adventure, Drama"
1,2024-05-26,Films (English),2,Thelma the Unicorn,NaN,17400000,1.6333,10700000.0,2,False,NaN,"Animation, Adventure, Comedy"
2,2024-05-26,Films (English),3,Ice Age: Dawn of the Dinosaurs,NaN,15100000,1.5833,9500000.0,1,False,NaN,"Animation, Adventure, Comedy"
3,2024-05-26,Films (English),4,Mother of the Bride,NaN,14000000,1.5000,9300000.0,3,False,NaN,"Comedy, Drama, Romance"
4,2024-05-26,Films (English),5,Disturbia,NaN,14100000,1.7500,8100000.0,2,False,NaN,"Crime, Drama, Mystery"
5,2024-05-26,Films (English),6,Sing 2,NaN,13900000,1.8333,7600000.0,10,False,NaN,"Animation, Adventure, Comedy"
6,2024-05-26,Films (English),7,Ice Age: Collision Course,NaN,11700000,1.6167,7200000.0,1,False,NaN,"Animation, Adventure, Comedy"
7,2024-05-26,Films (English),8,A Simple Favor,NaN,12200000,1.9500,6300000.0,1,False,NaN,"Comedy, Crime, Mystery"
8,2024-05-26,Films (English),9,Madame Web,NaN,11900000,1.9333,6200000.0,2,False,NaN,"Action, Adventure, Sci-Fi"
9,2024-05-26,Films (English),10,Security,NaN,5100000,1.5333,3300000.0,3,False,NaN,"Action, Crime, Thriller"


In [45]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        6080 non-null   object 
 1   category                    6080 non-null   object 
 2   weekly_rank                 6080 non-null   int64  
 3   show_title                  6080 non-null   object 
 4   season_title                2959 non-null   object 
 5   weekly_hours_viewed         6080 non-null   int64  
 6   runtime                     2000 non-null   float64
 7   weekly_views                2000 non-null   float64
 8   cumulative_weeks_in_top_10  6080 non-null   int64  
 9   is_staggered_launch         6080 non-null   bool   
 10  episode_launch_details      78 non-null     object 
 11  gender                      3349 non-null   object 
dtypes: bool(1), float64(2), int64(3), object(6)
memory usage: 528.6+ KB


In [46]:
# Generate a CSV file from the DataFrame
output_file_path = 'neflix_global_dataset_with_gender.csv'
df_global.to_csv(output_file_path, index=False)

In [27]:
count_with_gender= count_no_gender= 0
no_gender = []
with_gender = []
for title, genre in title_gender_dict.items():
    if title_gender_dict[title] != "":
        count_with_gender += 1
        with_gender.append(title)
    else:
        count_no_gender += 1
        no_gender.append(title)
        
print(f"Titulos com genero totalizam: {count_with_gender}")
print(f"Titulo sem genere totalizam: {count_no_gender}")

Titulos com genero totalizam: 1906
Titulo sem genere totalizam: 87


In [36]:
print("Títulos COM gênero")

unique_gender_values: list = []
    
for title, genre in title_gender_dict.items():
    if genre != "" and genre is not None:
        for type_genre in genre.split(","):
            if type_genre not in unique_gender_values:
                unique_gender_values.append(type_genre)

Títulos COM gênero


In [37]:
unique_gender_values

['Action',
 ' Adventure',
 ' Drama',
 'Animation',
 ' Comedy',
 'Comedy',
 ' Romance',
 'Crime',
 ' Mystery',
 ' Crime',
 ' Sci-Fi',
 ' Thriller',
 ' Action',
 'Drama',
 ' History',
 'Biography',
 'Horror',
 ' Fantasy',
 'Documentary',
 'Short',
 ' Family',
 ' Music',
 'Reality-TV',
 ' Short',
 'Adventure',
 ' Talk-Show',
 ' Horror',
 ' Biography',
 ' Sport',
 'Game-Show',
 ' Reality-TV',
 ' Western',
 'Thriller',
 ' War',
 'N/A',
 'Romance',
 'Fantasy',
 ' Musical',
 'Mystery',
 ' Game-Show',
 'Music',
 'Musical',
 ' News']

In [39]:
counting_none_gender = 0
for title, genre in title_gender_dict.items():
    if genre != "" and genre is None:
        counting_none_gender += 1
        print(f"titulo: {title}, genero: {genre}")
print(f"Contagem final de generos do tipo None: {counting_none_gender}")

titulo: The Law According to Lidia Poët, genero: None
titulo: Army of One, genero: None
titulo: The Woman King, genero: None
titulo: 2 Guns, genero: None
titulo: Jurassic World, genero: None
titulo: Mission: Impossible - Fallout, genero: None
titulo: Cirkus, genero: None
titulo: Squared Love All Over Again, genero: None
titulo: All the Places, genero: None
titulo: Re/Member, genero: None
titulo: Elite Squad, genero: None
titulo: An Inconvenient Love, genero: None
titulo: The Last Mercenary, genero: None
titulo: Red Rose, genero: None
titulo: The Walking Dead, genero: None
titulo: Ganglands, genero: None
titulo: Love to Hate You, genero: None
titulo: Eva Lasting, genero: None
titulo: You People, genero: None
titulo: 47 Meters Down: Uncaged, genero: None
titulo: True Spirit, genero: None
titulo: Bloodshot, genero: None
titulo: Thunivu (Hindi), genero: None
titulo: Infiesto, genero: None
titulo: Narvik, genero: None
titulo: Ginny & Georgia, genero: None
titulo: Full Swing, genero: None
ti

In [30]:
print("Títulos COM gênero")

unique_gender_values: list = []
    
for title, genre in title_gender_dict.items():
    if genre != "":
        print(60*"-")
        print(string_test.split(","))
        for type_genre in genre.split(","):
            if type_genre not in unique_gender_values:
                unique_gender_values.append(type_genre)
        print(f"título: {title}, gênero: {genre}")

Títulos COM gênero
------------------------------------------------------------
título: Atlas, gênero: Action, Adventure, Drama
------------------------------------------------------------
título: Thelma the Unicorn, gênero: Animation, Adventure, Comedy
------------------------------------------------------------
título: Ice Age: Dawn of the Dinosaurs, gênero: Animation, Adventure, Comedy
------------------------------------------------------------
título: Mother of the Bride, gênero: Comedy, Drama, Romance
------------------------------------------------------------
título: Disturbia, gênero: Crime, Drama, Mystery
------------------------------------------------------------
título: Sing 2, gênero: Animation, Adventure, Comedy
------------------------------------------------------------
título: Ice Age: Collision Course, gênero: Animation, Adventure, Comedy
------------------------------------------------------------
título: A Simple Favor, gênero: Comedy, Crime, Mystery
--------------

In [31]:
print("Títulos SEM gênero")
for title, genre in title_gender_dict.items():
    if genre == "":
        print(60*"-")
        print(f"título: {title}, gênero: {genre}")

Títulos SEM gênero
------------------------------------------------------------
título: In Good Hands 2, gênero: 
------------------------------------------------------------
título: Buying London, gênero: 
------------------------------------------------------------
título: The 8 Show, gênero: 
------------------------------------------------------------
título: Franco Escamilla: Ladies' man, gênero: 
------------------------------------------------------------
título: The Atypical Family, gênero: 
------------------------------------------------------------
título: Garouden: The Way of the Lone Wolf, gênero: 
------------------------------------------------------------
título: Maestro in Blue, gênero: 
------------------------------------------------------------
título: Partiu América, gênero: 
------------------------------------------------------------
título: Honeymoonish, gênero: 
------------------------------------------------------------
título: The Guardian of the Monarchs, g

In [6]:
x_df = df_global[df_global['episode_launch_details'].notna()].copy()

In [8]:
x_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78 entries, 35 to 1999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        78 non-null     object 
 1   category                    78 non-null     object 
 2   weekly_rank                 78 non-null     int64  
 3   show_title                  78 non-null     object 
 4   season_title                78 non-null     object 
 5   weekly_hours_viewed         78 non-null     int64  
 6   runtime                     78 non-null     float64
 7   weekly_views                78 non-null     float64
 8   cumulative_weeks_in_top_10  78 non-null     int64  
 9   is_staggered_launch         78 non-null     bool   
 10  episode_launch_details      78 non-null     object 
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 6.8+ KB


In [ ]:
is_staggered_launch

In [9]:
for column_name in df_global.columns:
    print(column_name)

week
category
weekly_rank
show_title
season_title
weekly_hours_viewed
runtime
weekly_views
cumulative_weeks_in_top_10
is_staggered_launch
episode_launch_details


# Dicionário de dados

### - week: Data do período semanal;
### - category: Se é filme ou programa de TV em inglês ou não;
### - weekly_rank: Rank semanal, mostra em que posição dos top 10 está o filme ou programa, recebe um inteiro que vai de um a dez;
### - show_title: Título do filme ou programa de TV;
### - season_title: Identifica a temporada do programa de TV;
### - weekly_hours_viewed: Horas semanais exibidas demandadas 
### - runtime: Tempo de duração do filme ou programa de TV;
### - weekly_views: Números de views semanais; 
### - cumulative_weeks_in_top_10: Número em que o programa se manteve; 
### - is_staggered_launch: 
### - episode_launch_details

# Transformações sugeridas a serem realizadas

## - Coluna Week tem que ficar em formato Date;

## - Deletar coluna de 'Runtime'. (Obs: Não vai acrescentar na análise e tem muitos dados faltantes);

## - Deletar a coluna 'season_title'. (Obs: A análise pode ser voltada para a preferência por título. Nessa coluna temos a informação do título e a temporada. A temporada é uma informação descartável no contexto de preferência de usuário se baseado no prisma por gênero do título);

# Transformações opcionais

## - Staggered Launch só faz sentido se fizermos uma análise mais apurada da audiência de alguns programas específicos com lançamentos por episódios paulatinos, uma vez que ela e a coluna de detalhes de episódio de lançamento (episode_launch_details) podem nos dar uma dica sobre esses detalhes;  (Deletar ou não deletar)?

## Observações

### - Runtime: Tempo de duração do filme ou do episódio da série (Não consegui descobrir se é em minutos ou horas);

### - is_staggered_launch: Variável booleana que dá detalhes se os episódios de um programa são liberados paulatinamente e não de uma só vez;

## **Transformação**

### **Exclusão de Colunas**

In [ ]:
#Exclusão de colunas (números elevados de valores nulos)
lista = ['episode_launch_details','season_title','weekly_views','runtime','is_staggered_launch']

for i in lista:
  df.drop(i, axis=1, inplace=True)

In [ ]:
# Renomeando colunas para tradução
df.rename(columns={'week' : 'semana',
                   'category' : 'categoria',
                   'weekly_rank' : 'classificaçao_semanal',
                   'show_title' : 'titulo_do_programa/show',
                   'season_title' : 'titulo da temporada',
                   'weekly_hours_viewed' : 'horas_visualizadas',
                   'cumulative_weeks_in_top_10' : 'semanas_acumuladas_top10'}, inplace=True)

### **Verificação de valores nulos**

In [ ]:
# Verificando valores null
df.isnull().sum()

semana                      0
categoria                   0
classificaçao_semanal       0
titulo_do_programa/show     0
horas_visualizadas          0
semanas_acumuladas_top10    0
dtype: int64

In [ ]:
# Verificando valores nan
df.isna().sum()

semana                      0
categoria                   0
classificaçao_semanal       0
titulo_do_programa/show     0
horas_visualizadas          0
semanas_acumuladas_top10    0
dtype: int64

### **Valores unicos e dados duplicados (REVISAR)**

Como em nossa tabela todos os valores de todas as colunas se repetem eventualmente, é necessário fazer uma verificação de valores duplicados de um mesmo título na mesma semana.

## **Conversão de String para Date Time**

In [ ]:
df['semana'] = pd.to_datetime(df['semana'], infer_datetime_format=True)

<ipython-input-31-7c84dc2c8b97>:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['semana'] = pd.to_datetime(df['semana'], infer_datetime_format=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   semana                    6080 non-null   datetime64[ns]
 1   categoria                 6080 non-null   object        
 2   classificaçao_semanal     6080 non-null   int64         
 3   titulo_do_programa/show   6080 non-null   object        
 4   horas_visualizadas        6080 non-null   int64         
 5   semanas_acumuladas_top10  6080 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 285.1+ KB


In [ ]:
df['mes'] = df['semana'].dt.month
df['ano'] = df['semana'].dt.year
df['dia'] = df['semana'].dt.day

### **Traduzir valores das colunas ???????**